In [1]:
import pandas as pd
import numpy as np

# Load dim
target_path = 'Data/Target/'

dim_customer = pd.read_csv(target_path + 'DimCustomer.csv')
dim_city = pd.read_csv(target_path + 'DimCity.csv')
dim_date = pd.read_csv(target_path + 'DimDate.csv')
dim_orderStatus = pd.read_csv(target_path + 'DimOrderStatus.csv')
dim_product = pd.read_csv(target_path + 'DimProduct.csv')
dim_returnReason = pd.read_csv(target_path + 'DimReturnReason.csv')
dim_salesAgent = pd.read_csv(target_path + 'DimSalesAgent.csv') 
dim_state = pd.read_csv(target_path + 'DimState.csv')

dim_date['DateKey'] = dim_date['DateKey'].astype(str)

#load original data
dataset_path = r'Data\Original\Dataset.xlsx'
dict_df = pd.read_excel(dataset_path, 
                            sheet_name=[i for i in range(3)])

sales = pd.concat([dict_df.get(i) for i in range(len(dict_df))])
sales.shape


(3000, 14)

In [2]:
sales['Return Reason'].replace(np.nan, 'Dont have', inplace=True)
sales.head()

C:\Users\TOAN.000\AppData\Local\Temp\ipykernel_17788\1367976656.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sales['Return Reason'].replace(np.nan, 'Dont have', inplace=True)


,Order ID,Customer Last Name,Customer First Name,Customer ID,Product Category,Order Date,Quantity Ordered,Order Status,State,Manufacturer Price,Sale Price,Total Profit (GMROI),Sales Agent,Return Reason
0,1001,Hughes,Lewis,1234,Clothing,2022-10-11,16,Delivered,LA,1485,1880,395,Kelly,Dont have
1,1002,Gomez,Ali,1235,Home & Kitchen,2022-10-09,11,Delivered,LA,1310,1602,292,Brian,Dont have
2,1003,Beck,Mila,1236,Personal Care,2022-10-21,4,Shipped,GA,800,891,91,Terrence,Dont have
3,1004,Juarez,Ryleigh,1237,Personal Care,2022-10-26,20,Cancelled,NC,1190,1462,272,Brian,Defective
4,1005,Preston,Leonidas,1238,Electronics,2022-10-22,9,Shipped,GA,470,525,55,Daniel,Dont have


In [3]:
print(sales.shape)

fact_df = pd.merge(sales, dim_customer['CustomerKey'], 
                   how='left', 
                   left_on='Customer ID', 
                   right_on='CustomerKey').drop(
                       columns=['Customer ID', 
                                'Customer Last Name', 
                                'Customer First Name'])

print(fact_df.shape)

fact_df = pd.merge(fact_df, dim_state, 
                   how='left', 
                   left_on='State', 
                   right_on='State').drop(columns=['State'])

print(fact_df.shape)

fact_df = pd.merge(fact_df, dim_orderStatus,
                how='left',
                left_on='Order Status', 
                right_on='OrderStatusName').drop(columns=['OrderStatusName', 'Order Status'])

print(fact_df.shape)

fact_df = pd.merge(fact_df, dim_returnReason,
                   how='left',
                   left_on='Return Reason',
                   right_on='ReturnReasonName').drop(columns=['ReturnReasonName', 'Return Reason'])

print(fact_df.shape)

fact_df = pd.merge(fact_df, dim_salesAgent, 
                   how='left',
                   left_on='Sales Agent',
                   right_on='SaleAgentName').drop(columns=['SaleAgentName', 'Sales Agent'])

print(fact_df.shape)

fact_df = pd.merge(fact_df, dim_product,
                how='left',
                left_on='Product Category',
                right_on='ProductCatalogName').drop(columns=['ProductCatalogName', 'Product Category'])

print(fact_df.shape)

(3000, 14)
(3000, 12)
(3000, 12)
(3000, 12)
(3000, 12)
(3000, 12)
(3000, 12)


In [4]:
fact_df['Order Date'] = fact_df['Order Date'].apply(lambda x: x.strftime('%Y%m%d'))

fact_df = pd.merge(fact_df, dim_date['DateKey'], 
                left_on='Order Date',
                right_on='DateKey').drop(columns=['Order Date'])

In [5]:
fact_df.shape
# sales.shape

(3000, 12)

In [6]:
fact_df.to_csv(target_path + 'FactOrder.csv', index=False)

In [7]:
fact_df = pd.merge(sales, dim_customer['CustomerKey'], 
                   how='left', 
                   left_on='Customer ID', 
                   right_on='CustomerKey').drop(
                       columns=['Customer ID', 
                                'Customer Last Name', 
                                'Customer First Name'])

In [8]:
fact_df

,Order ID,Product Category,Order Date,Quantity Ordered,Order Status,State,Manufacturer Price,Sale Price,Total Profit (GMROI),Sales Agent,Return Reason,CustomerKey
0,1001,Clothing,2022-10-11,16,Delivered,LA,1485,1880,395,Kelly,Dont have,1234
1,1002,Home & Kitchen,2022-10-09,11,Delivered,LA,1310,1602,292,Brian,Dont have,1235
2,1003,Personal Care,2022-10-21,4,Shipped,GA,800,891,91,Terrence,Dont have,1236
3,1004,Personal Care,2022-10-26,20,Cancelled,NC,1190,1462,272,Brian,Defective,1237
4,1005,Electronics,2022-10-22,9,Shipped,GA,470,525,55,Daniel,Dont have,1238
...,...,...,...,...,...,...,...,...,...,...,...,...
2995,3996,Pets,2022-12-23,12,Shipped,LA,1465,1865,400,Daniel,Dont have,2227
2996,3997,Books,2022-12-25,4,Cancelled,GA,890,1050,160,Kelly,Quality Issue,2228
2997,3998,Pets,2022-12-23,20,Shipped,LA,825,1059,234,Daniel,Dont have,2229
2998,3999,Sports & Outdoors,2022-12-14,16,Shipped,LA,875,1116,241,Priscilla,Dont have,2230


In [9]:
dim_customer['CustomerKey'].value_counts()

CustomerKey
1234    1
1904    1
1891    1
1892    1
1893    1
       ..
1571    1
1572    1
1573    1
1574    1
2231    1
Name: count, Length: 998, dtype: int64